Build a list for Training, Validation and testing dataset 

In [1]:
import sys
sys.path.append('/workspace/Documents')
import os
import numpy as np
import pandas as pd
import nibabel as nb
import Diffusion_for_CT_motion.utils.functions_collection as ff

/usr/local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# for new data colleted 202404

In [3]:
import random
data_path = '/mnt/camca_NAS/Portable_CT_data'
l = ff.find_all_target_files(['*/*'],os.path.join(data_path,'simulations_202404/simulated_partial_motion_v2'))
print(len(l))

patient_sheet = pd.read_excel(os.path.join(data_path,'Patient_list', 'NEW_CT_concise_collected_fixed_static_edited.xlsx'),dtype={'Patient_ID': str, 'Patient_subID': str})
patient_sheet['use'] = patient_sheet['use'].fillna(0)
patient_sheet = patient_sheet[(patient_sheet['use'] != 0) & (patient_sheet['use'] != 'no')]
print('patient sheet len: ', len(patient_sheet))

assert len(l) == len(patient_sheet)

# randomly shuffle l and split into five groups 

# Set a random seed
random.seed(1)

# if os.path.isfile(os.path.join('/mnt/camca_NAS/diffusion_ct_motion/data','Patient_list', 'Patient_list_new_data_batch.xlsx')) == 0:
#     # Shuffle the list
#     a = np.arange(0,len(patient_sheet),1)
#     random.shuffle(a)
#     print(a)

#     # make a as a new column in patient_sheet
#     patient_sheet['random_index'] = a

#     # split into 5 groups based on the random index
#     patient_sheet['batch'] = 0
#     patient_sheet.loc[patient_sheet['random_index'] < 20, 'batch'] = 0
#     patient_sheet.loc[(patient_sheet['random_index'] >= 20) & (patient_sheet['random_index'] < 40), 'batch'] = 1
#     patient_sheet.loc[(patient_sheet['random_index'] >= 40) & (patient_sheet['random_index'] < 60), 'batch'] = 2
#     patient_sheet.loc[(patient_sheet['random_index'] >= 60) & (patient_sheet['random_index'] < 80), 'batch'] = 3
#     patient_sheet.loc[patient_sheet['random_index'] >= 80, 'batch'] = 4

#     # save the patient_sheet
#     patient_sheet.to_excel(os.path.join('/mnt/camca_NAS/diffusion_ct_motion/data','Patient_list', 'Patient_list_new_data_batch.xlsx'),index=False)

100
patient sheet len:  100


In [4]:
data = pd.read_excel(os.path.join('/mnt/camca_NAS/diffusion_ct_motion/data','Patient_list', 'Patient_list_new_data_batch.xlsx'),dtype={'Patient_ID': str, 'Patient_subID': str})
Result = []
random_list = np.arange(1,6,1)
slice_list = [[0,60]]
slice_total_list = []


for i in range(0,data.shape[0]):
    case = data.iloc[i]
    patient_id = str(case['Patient_ID'])
    patient_subid = str(case['Patient_subID'])
    batch = case['batch']
    
    if batch == 4:
        train = 'test'
    else:
        train = 'train'
       
    # motion-free
    motion_free_file = os.path.join(data_path,'simulations_202404/simulated_partial_motion_v2',patient_id,patient_subid,'static/image_data/recon_resample.nii.gz')
    print(motion_free_file)
    assert os.path.isfile(motion_free_file) == 1

    # motion-free downsampled and cropped
    motion_free_ds_file = ''

    for j in random_list:
        motion_name = 'random_'+ str(j)
        print(i, patient_id, patient_subid, motion_name)

        # motion image
        motion_corrupted_file = os.path.join(data_path,'simulations_202404/simulated_partial_motion_v2',patient_id,patient_subid,motion_name,'image_data/recon_resample.nii.gz')
        assert os.path.isfile(motion_corrupted_file) == 1
        
        # motion image downsampled and cropped
        motion_corrupted_ds_file = ''
        
        # motion parameters
        motion_parameter_file = os.path.join(data_path,'simulations_202404/simulated_partial_motion_v2',patient_id,patient_subid,motion_name,'motion_parameters.npy')
            
      
        for s in slice_list:
            start_slice = s[0]
            end_slice = s[1]
            Result.append([train, batch, patient_id, patient_subid, motion_name, start_slice, end_slice, motion_free_file, motion_free_ds_file, motion_corrupted_file, 
                                motion_corrupted_ds_file, motion_parameter_file])


column_list = ['group','batch','PatientID','AccessionNumber','MotionName','PAR_start_slice', 'PAR_end_slice', 'MotionFree_File', 'MotionFreeDs_File',
                'Motion_File', 'MotionDs_File', 'MotionParam']
df = pd.DataFrame(Result, columns = column_list)
df.to_excel(os.path.join('/mnt/camca_NAS/diffusion_ct_motion/data','Patient_list/Patient_list_train_test_simulated_partial_motion_v2.xlsx'),index=False)


/mnt/camca_NAS/Portable_CT_data/simulations_202404/simulated_partial_motion_v2/00194824/0000455659/static/image_data/recon_resample.nii.gz
0 00194824 0000455659 random_1
0 00194824 0000455659 random_2
0 00194824 0000455659 random_3
0 00194824 0000455659 random_4
0 00194824 0000455659 random_5
/mnt/camca_NAS/Portable_CT_data/simulations_202404/simulated_partial_motion_v2/00064366/0000455657/static/image_data/recon_resample.nii.gz
1 00064366 0000455657 random_1
1 00064366 0000455657 random_2
1 00064366 0000455657 random_3
1 00064366 0000455657 random_4
1 00064366 0000455657 random_5
/mnt/camca_NAS/Portable_CT_data/simulations_202404/simulated_partial_motion_v2/00214934/0000455653/static/image_data/recon_resample.nii.gz
2 00214934 0000455653 random_1
2 00214934 0000455653 random_2
2 00214934 0000455653 random_3
2 00214934 0000455653 random_4
2 00214934 0000455653 random_5
/mnt/camca_NAS/Portable_CT_data/simulations_202404/simulated_partial_motion_v2/00148556/0000455634/static/image_data/r